In [ ]:
#! pip install optuna
#! pip install ray[tune]

In [1]:
from utils import getData
from datasets import load_dataset 
import pandas as pd 
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import load_dataset
import pandas as pd 
import numpy as np
import torch
import matplotlib.pyplot as plt
from collections import Counter
from datasets import load_metric
from transformers import EarlyStoppingCallback

In [2]:
f1 = load_metric("f1")
recall = load_metric("recall")
precision =  load_metric("precision")
def preprocess_function(examples, tok):
    return tok(examples["text"], truncation=True, max_length=512)


def compute_metrics(p):    
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    metric = f1.compute(predictions=predictions, references=labels, average="macro")
    metric.update(recall.compute(predictions=predictions, references=labels, average="macro"))
    metric.update(precision.compute(predictions=predictions, references=labels, average="macro"))
    return metric
data = getData(sub_task = f"A", return_type = "dataset", pre_proccessed = True)
tokenizer = AutoTokenizer.from_pretrained("UBC-NLP/MARBERTv2")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
tokenized_data = data.map(preprocess_function,fn_kwargs = {'tok':tokenizer}, batched=True)

Pre-Processed


Using custom data configuration default-87434f9d42f04810
Reusing dataset csv (/home/ahmad.shapiro/.cache/huggingface/datasets/csv/default-87434f9d42f04810/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached processed dataset at /home/ahmad.shapiro/.cache/huggingface/datasets/csv/default-87434f9d42f04810/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-cc20df7b5e081abc.arrow
Loading cached processed dataset at /home/ahmad.shapiro/.cache/huggingface/datasets/csv/default-87434f9d42f04810/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-6d8f0bdb202bf214.arrow


In [3]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained("UBC-NLP/MARBERTv2", num_labels=2)

In [4]:
training_args = TrainingArguments(
    output_dir="./Checkpoints/MarBertv2_hyperparams",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy="epoch",
    save_strategy = "epoch",
    metric_for_best_model="f1",
    num_train_epochs=20,
    weight_decay=0.01,
    load_best_model_at_end=True,
    group_by_length = True, 
    )
trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

loading configuration file https://huggingface.co/UBC-NLP/MARBERTv2/resolve/main/config.json from cache at /home/ahmad.shapiro/.cache/huggingface/transformers/ff060a5035cde771cddd0649d04ca5403ad27e4dd88e31cdb10ce3a3169c7eea.8480f475cb3f32e19cacdd9bac8fc808a24bd5d183a66fbcb4525caebf3a97a7
Model config BertConfig {
  "_name_or_path": "UBC-NLP/MARBERTv2",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "pos

In [5]:
def my_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True),
        "weight_decay" : trial.suggest_float("weight_decay", 1e-10, 1e-4, log=True),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 1, 20),
        "seed": trial.suggest_int("seed", 1, 50),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [4, 8, 16, 32]),
    }

In [ ]:
best_run = trainer.hyperparameter_search(n_trials=30, direction="maximize",  hp_space=my_hp_space)

[I 2022-03-23 22:25:30,414] A new study created in memory with name: no-name-deba6045-48cb-434a-a392-07493cf517db
Trial:
loading configuration file https://huggingface.co/UBC-NLP/MARBERTv2/resolve/main/config.json from cache at /home/ahmad.shapiro/.cache/huggingface/transformers/ff060a5035cde771cddd0649d04ca5403ad27e4dd88e31cdb10ce3a3169c7eea.8480f475cb3f32e19cacdd9bac8fc808a24bd5d183a66fbcb4525caebf3a97a7
Model config BertConfig {
  "_name_or_path": "UBC-NLP/MARBERTv2",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention

Epoch,Training Loss,Validation Loss,F1,Recall,Precision
1,0.712900,0.667205,0.405431,0.500000,0.340945
2,0.689000,0.626864,0.405431,0.500000,0.340945
3,0.683700,0.625857,0.405431,0.500000,0.340945
4,0.661900,0.625711,0.405431,0.500000,0.340945
5,0.655400,0.627398,0.405431,0.500000,0.340945
6,0.652000,0.628660,0.405431,0.500000,0.340945
7,0.653000,0.629114,0.405431,0.500000,0.340945


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1270
  Batch size = 4
/home/ahmad.shapiro/anaconda3/envs/pytorch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to ./Checkpoints/MarBertv2_hyperparams/run-0/checkpoint-1111
Configuration saved in ./Checkpoints/MarBertv2_hyperparams/run-0/checkpoint-1111/config.json
Model weights saved in ./Checkpoints/MarBertv2_hyperparams/run-0/checkpoint-1111/pytorch_model.bin
tokenizer config file saved in ./Checkpoints/MarBertv2_hyperpara

[I 2022-03-23 22:34:42,909] Trial 0 finished with value: 1.2463755935002507 and parameters: {'learning_rate': 0.001768155603261361, 'weight_decay': 5.05385793781321e-09, 'num_train_epochs': 7, 'seed': 40, 'per_device_train_batch_size': 8}. Best is trial 0 with value: 1.2463755935002507.
Trial:
loading configuration file https://huggingface.co/UBC-NLP/MARBERTv2/resolve/main/config.json from cache at /home/ahmad.shapiro/.cache/huggingface/transformers/ff060a5035cde771cddd0649d04ca5403ad27e4dd88e31cdb10ce3a3169c7eea.8480f475cb3f32e19cacdd9bac8fc808a24bd5d183a66fbcb4525caebf3a97a7
Model config BertConfig {
  "_name_or_path": "UBC-NLP/MARBERTv2",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_po

Epoch,Training Loss,Validation Loss,F1,Recall,Precision
1,0.693100,0.640827,0.405431,0.500000,0.340945
2,0.691300,0.628014,0.405431,0.500000,0.340945
3,0.660100,0.625480,0.405431,0.500000,0.340945
4,0.655200,0.625437,0.405431,0.500000,0.340945
5,0.661100,0.625614,0.405431,0.500000,0.340945
6,0.653500,0.626113,0.405431,0.500000,0.340945
7,0.655900,0.626580,0.405431,0.500000,0.340945


...

Epoch,Training Loss,Validation Loss,F1,Recall,Precision
1,No log,0.323559,0.847319,0.857859,0.839642
2,0.323400,0.610154,0.770757,0.819273,0.777046
3,0.323400,0.459374,0.841625,0.838712,0.844797
4,0.121800,0.595065,0.848981,0.861738,0.840308


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1270
  Batch size = 4
Saving model checkpoint to ./Checkpoints/MarBertv2_hyperparams/run-18/checkpoint-278
Configuration saved in ./Checkpoints/MarBertv2_hyperparams/run-18/checkpoint-278/config.json
Model weights saved in ./Checkpoints/MarBertv2_hyperparams/run-18/checkpoint-278/pytorch_model.bin
tokenizer config file saved in ./Checkpoints/MarBertv2_hyperparams/run-18/checkpoint-278/tokenizer_config.json
Special tokens file saved in ./Checkpoints/MarBertv2_hyperparams/run-18/checkpoint-278/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not exp

Epoch,Training Loss,Validation Loss,F1,Recall,Precision
1,0.410500,0.334527,0.847719,0.844074,0.851763
2,0.249100,0.456978,0.844326,0.856210,0.836075
3,0.157100,0.638135,0.841777,0.853817,0.833495
4,0.094300,0.713403,0.846316,0.849770,0.843193
5,0.053900,0.754401,0.851388,0.858682,0.845488


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1270
  Batch size = 4
Saving model checkpoint to ./Checkpoints/MarBertv2_hyperparams/run-19/checkpoint-556
Configuration saved in ./Checkpoints/MarBertv2_hyperparams/run-19/checkpoint-556/config.json
Model weights saved in ./Checkpoints/MarBertv2_hyperparams/run-19/checkpoint-556/pytorch_model.bin
tokenizer config file saved in ./Checkpoints/MarBertv2_hyperparams/run-19/checkpoint-556/tokenizer_config.json
Special tokens file saved in ./Checkpoints/MarBertv2_hyperparams/run-19/checkpoint-556/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not exp

Epoch,Training Loss,Validation Loss
